In [158]:
import scanpy as sc 
import pandas as pd
import os
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 70)  # show all columns
pd.set_option('display.width', None)        # don’t wrap columns


In [161]:
adata= sc.read_h5ad('/lustre/scratch126/cellgen/haniffa/sm54/BoneAtlasProject/data/bone_atlas_filtered_with_metadata.h5ad')

In [162]:
adata

AnnData object with n_obs × n_vars = 2493304 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S

In [163]:
samples=adata[adata.obs['Multiplexed']=="Yes"]
samples

View of AnnData object with n_obs × n_vars = 271533 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_

In [165]:
samples_ids=list(samples.obs['Sample_ID'].unique())
(samples_ids)

['HCA_BN_F13177434',
 'HCA_BN_F13177435',
 'HCA_BN_F13177436',
 'HCA_BN_F12808058',
 'HCA_BN_F12808059',
 'HCA_BN_F12808060',
 'HCA_BN_F12627471',
 'HCA_BN_F12922485',
 'HCA_BN_F12627472',
 'HCA_BN_F12627470',
 'HCA_BN_F12922487',
 'HCA_BN_F12627473',
 'HCA_BN_F13109714',
 'HCA_BN_F13109715',
 'HCA_BN_F13109718',
 'HCA_BN_F13109719']

In [166]:
import scanpy as sc
import pandas as pd
import os

# ─── Inputs ────────────────────────────────────────────────────────────────
souporcell_base = "/nfs/cellgeni/tickets/tic-3721/Souporcell" 
# list all your 26 sample IDs here (exact folder names before ".souporcell.out")

# ─── 1) Prepare the annotation column ───────────────────────────────────────
# Default to “unassigned” for every cell
adata.obs["demuxed_sample"] = "unassigned"
# we'll make it a category at the end


# Start Donor_Demultiplexed as a copy of your existing Donor_ID
adata.obs["Donor_Demultiplexed"] = adata.obs["Donor_ID"].copy().astype(str)

# ─── 2) Loop over each sample’s Souporcell output ─────────────────────────
all_dfs = []
for sid in samples_ids:
    folder  = os.path.join(souporcell_base, f"{sid}.souporcell.out")
    ct_file = os.path.join(folder, "clusters.tsv")
    if not os.path.isfile(ct_file):
        print(f"⚠️  clusters.tsv not found for {sid}, skipping.")
        continue

    # Read only the columns we need
    df = pd.read_csv(
        ct_file,
        sep="\t",
        usecols=["barcode", "assignment", "status"],
        dtype={"barcode":str, "assignment":str, "status":str}
    )
    # Add Sample_ID and build a unique Cell_ID that matches adata.obs_names
    df["Sample_ID"] = sid
    df["Donor"] = df["assignment"] + "-" + sid
    df["Cell_ID"]   = df["barcode"] + "-" + sid
    df = df.set_index("Cell_ID")

    # Overwrite Donor_Demultiplexed for these demuxed cells:
    # here we include all cells, or you could filter df["status"] == "singlet"
    #
    valid_cells = df.index
    adata.obs.loc[
        adata.obs_names.intersection(valid_cells),
        "Donor_Demultiplexed"
    ] = df.loc[
        adata.obs_names.intersection(valid_cells),
        "Donor"
    ]

    all_dfs.append(df)

# ─── 3) Finalize the new column as categorical ─────────────────────────────
adata.obs["Donor_Demultiplexed"] = \
    adata.obs["Donor_Demultiplexed"].astype("category")

# ─── 4) Concatenate all per-sample dfs into one DataFrame ───────────────────
concat_df = pd.concat(all_dfs, axis=0)
print("Concatenated souporcell assignments:")
print(concat_df.head())


Concatenated souporcell assignments:
                                            barcode   status assignment  \
Cell_ID                                                                   
AAACAGCCAACTGGGA-HCA_BN_F13177434  AAACAGCCAACTGGGA  singlet          0   
AAACAGCCAATAACCT-HCA_BN_F13177434  AAACAGCCAATAACCT  singlet          1   
AAACAGCCACAGGAAT-HCA_BN_F13177434  AAACAGCCACAGGAAT  singlet          0   
AAACAGCCAGTAAAGC-HCA_BN_F13177434  AAACAGCCAGTAAAGC  singlet          1   
AAACAGCCAGTTGCGT-HCA_BN_F13177434  AAACAGCCAGTTGCGT  singlet          1   

                                          Sample_ID               Donor  
Cell_ID                                                                  
AAACAGCCAACTGGGA-HCA_BN_F13177434  HCA_BN_F13177434  0-HCA_BN_F13177434  
AAACAGCCAATAACCT-HCA_BN_F13177434  HCA_BN_F13177434  1-HCA_BN_F13177434  
AAACAGCCACAGGAAT-HCA_BN_F13177434  HCA_BN_F13177434  0-HCA_BN_F13177434  
AAACAGCCAGTAAAGC-HCA_BN_F13177434  HCA_BN_F13177434  1-HCA_BN_F1317

In [178]:
missing_barcodes= (set(samples.obs['Cell_ID']) -set(concat_df.index))

In [183]:
missing_dmx= samples[samples.obs['Cell_ID'].isin(missing_barcodes)]

In [71]:
concat_df.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_multiplexed_barcodes.csv')

In [25]:
adata_old= sc.read_h5ad('/nfs/team298/sm54/skull_data_sharing/PCW13_Calvaria_with_Annotations.h5ad')

In [180]:
adata_old.obs["barcode_raw"] = adata_old.obs['barcode'].str.rsplit('-', n=1).str[0]
adata_old.obs['Sample_ID'] = adata_old.obs['sample_id'].str.split('_and_').str[0]
adata_old.obs["Cell_ID"]=adata_old.obs["barcode_raw"].astype(str) + '-' + adata_old.obs["Sample_ID"].astype(str) 

In [156]:
adata_old.obs["barcode_raw"] = adata_old.obs['barcode'].str.rsplit('-', n=1).str[0]
adata_old.obs['Sample_ID'] = adata_old.obs['sample_id'].str.split('_and_').str[0]


In [181]:
adata_old.obs.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/pcw13_skull_metadata.csv')

In [3]:
import scanpy as sc 
skull= sc.read_h5ad('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/datasets/Whole_RNA_website.h5ad')

In [9]:
skull.obs.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/to_et_al_2024_metadata.csv')

In [157]:
skull_metadata=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/to_et_al_2024_metadata.csv', index_col=0)
skull_metadata['barcode_raw']=skull_metadata.index.str.rsplit('-', n=1).str[1]
skull_metadata['Sample_ID'] = skull_metadata.index.str.split('_and_').str[0]
skull_metadata['Cell_ID'] = skull_metadata['barcode_raw'].astype(str).copy() + '-' + skull_metadata['Sample_ID'].astype(str).copy()


In [24]:
skull_metadata.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/to_et_al_2024_metadata.csv')

In [199]:
import pandas as pd 
skull_metadata= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/to_et_al_2024_metadata.csv', index_col=0)
pcw13_barcodes=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/pcw13_skull_metadata.csv')
concat_df= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_multiplexed_barcodes.csv')

In [200]:
concat_df

,Cell_ID,barcode,status,assignment,Sample_ID,Donor
0,AAACAGCCAACTGGGA-HCA_BN_F13177434,AAACAGCCAACTGGGA,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434
1,AAACAGCCAATAACCT-HCA_BN_F13177434,AAACAGCCAATAACCT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434
2,AAACAGCCACAGGAAT-HCA_BN_F13177434,AAACAGCCACAGGAAT,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434
3,AAACAGCCAGTAAAGC-HCA_BN_F13177434,AAACAGCCAGTAAAGC,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434
4,AAACAGCCAGTTGCGT-HCA_BN_F13177434,AAACAGCCAGTTGCGT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434
...,...,...,...,...,...,...
191955,TTTGTGAAGGGTGGAT-HCA_BN_F13109719,TTTGTGAAGGGTGGAT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719
191956,TTTGTGGCAGAAATTG-HCA_BN_F13109719,TTTGTGGCAGAAATTG,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719
191957,TTTGTTGGTGATGGCT-HCA_BN_F13109719,TTTGTTGGTGATGGCT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719
191958,TTTGTTGGTTCAAGCA-HCA_BN_F13109719,TTTGTTGGTTCAAGCA,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719


In [18]:
skull_metadata["Donor_ID"]=skull_metadata["brc_code"].copy()
skull_metadata["Anatomical_Site"]=skull_metadata["anatomical_site"].copy()
skull_metadata["Age"]=skull_metadata["pcw"].copy()
pcw13_barcodes["Donor_ID"] = "Donor_PCW13"
pcw13_barcodes['Sample_ID'] =pcw13_barcodes['sample_id'].str.split('_and_').str[0]


In [23]:
pcw13_barcodes.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/pcw13_skull_metadata.csv')

In [20]:
pcw13_barcodes['Cell_ID'] =pcw13_barcodes['barcode_raw'].astype(str).copy() + '-' + pcw13_barcodes['Sample_ID'].astype(str).copy()



In [ ]:
import scanpy as sc 
adata= sc.read_h5ad('/nfs/team298/sm54/skull_data_sharing/skull_object_with_annotations_with_raw_counts.h5ad')
# 1) Make sure it’s stringly-typed
adata.obs["Donor"] = adata.obs["Donor"].astype(str)

# 2) Remove any trailing “.0”
adata.obs["Donor"] = adata.obs["Donor"].str.replace(r"\.0$", "", regex=True)

# 3) Collapse all Sst2-BON-0-FO variants into one name
adata.obs["Donor"] = adata.obs["Donor"].str.replace(
    r"^Sst2-BON-0-FO.*$", 
    "Sst2-BON-0-FO-6to8", 
    regex=True
)

# 4) Check your work
print(adata.obs["Donor"].value_counts())


In [ ]:
adata.obs["barcode_raw"] = adata.obs['barcode'].str.rsplit('-', n=1).str[0]
adata.obs['Sample_ID'] = adata.obs['Run_ID'].str.split('_and_').str[0]
adata.obs["Cell_ID"]=adata.obs["barcode_raw"].astype(str) + '-' + adata.obs["Sample_ID"].astype(str) 

In [ ]:
adata.obs.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/skull_metadata_old_version_2.csv')

# Make Final Demultiplexed dataframe to add to the anndata object 

In [39]:
import pandas as pd 
skull_metadata=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/to_et_al_2024_metadata.csv', index_col=0)
pcw13_barcodes=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/pcw13_skull_metadata.csv')
concat_df= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_multiplexed_barcodes.csv')
skull_metadata2=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/skull_metadata_old_version_2.csv')

/tmp/ipykernel_3212985/3434238423.py:5: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  skull_metadata2=pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/old_annotations/skull_metadata_old_version_2.csv')


In [40]:
len(set(skull_metadata["Cell_ID"]) & set(concat_df["Cell_ID"]))

84157

In [41]:
len(set(pcw13_barcodes["Cell_ID"]) & set(concat_df["Cell_ID"]))

19561

In [42]:
len(set(skull_metadata2["Cell_ID"]) & set(concat_df["Cell_ID"]))

164020

In [43]:
len(set(skull_metadata2["Cell_ID"]) & set(concat_df["Cell_ID"]))

164020

In [44]:
cols_to_keep = ['Sample_ID', 'Cell_ID', 'Donor_ID','pcw']
existing = [c for c in cols_to_keep if c in pcw13_barcodes.columns]
pcw13_barcodes_2 = pcw13_barcodes[existing]

In [45]:
skull_metadata2.columns

Index(['cell_id', 'doublet_scores', 'predicted_doublets', 'sample_id',
       'barcode', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'Organ', 'Run_ID',
       'Technology', 'QC', 'cell_id.1', 'Donor', 'PCW', '_scvi_batch',
       '_scvi_labels', 'ANATOMICAL_SITE', 'YS_predicted_labels',
       'YS_over_clustering', 'YS_majority_voting', 'YS_conf_score',
       'WE_predicted_labels', 'WE_over_clustering', 'WE_majority_voting',
       'WE_conf_score', 'barcode_raw', 'Sample_ID', 'Cell_ID'],
      dtype='object')

In [46]:
cols_to_keep = ['Sample_ID', 'Cell_ID', 'Donor','ANATOMICAL_SITE',"PCW"]
existing = [c for c in cols_to_keep if c in skull_metadata2.columns]
skull_metadata22 = skull_metadata2[existing]

In [47]:
cols_to_keep = ['Sample_ID', 'Cell_ID', 'Celltype_fig1','pcw','brc_code','anatomical_site']
existing = [c for c in cols_to_keep if c in skull_metadata.columns]
skull_metadata2 = skull_metadata[existing]

In [141]:
import pandas as pd

# Assume:
# concatenated_df – your 160 K–cell master table, with a column “Cell_ID”
# df2 – metadata for ~80 K cells, with “Cell_ID” + its extra cols
# df3 – metadata for ~20 K cells, with “Cell_ID” + its extra cols

# 1) Merge df2 onto the master
merged = pd.merge(
    concat_df,
    pcw13_barcodes_2,
    on="Cell_ID",
    how="left",         # preserves all rows from concatenated_df
    suffixes=("", "_pcw13")  # avoids name collisions, if any
)

# 2) Merge df3 onto that result
merged = pd.merge(
    merged,
    skull_metadata2,
    on="Cell_ID",
    how="left",
    suffixes=("", "_published")
)

merged = pd.merge(
    merged,
    skull_metadata22,
    on="Cell_ID",
    how="left",
    suffixes=("", "_old_anno")
)


# (Optional) Inspect the shape and a few rows:
(merged.shape)    # should be (160000, original_cols + df2_cols + df3_cols)
(merged.head())


,Cell_ID,barcode,status,assignment,Sample_ID,Donor,Sample_ID_pcw13,Donor_ID,pcw,Sample_ID_published,Celltype_fig1,pcw_published,brc_code,anatomical_site,Sample_ID_old_anno,Donor_old_anno,ANATOMICAL_SITE,PCW
0,AAACAGCCAACTGGGA-HCA_BN_F13177434,AAACAGCCAACTGGGA,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3
1,AAACAGCCAATAACCT-HCA_BN_F13177434,AAACAGCCAATAACCT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,DermFIB1,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2
2,AAACAGCCACAGGAAT-HCA_BN_F13177434,AAACAGCCACAGGAAT,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3
3,AAACAGCCAGTAAAGC-HCA_BN_F13177434,AAACAGCCAGTAAAGC,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACAGCCAGTTGCGT-HCA_BN_F13177434,AAACAGCCAGTTGCGT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,HHIP+PreOB,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2


In [142]:
pcw13=merged[merged['pcw']==13.0]
pcw13["brc_code"].value_counts()

Series([], Name: count, dtype: int64)

In [143]:
import pandas as pd

# 1) Coerce both age columns to float
merged['pcw'] = pd.to_numeric( merged['pcw'],            errors='coerce' ).astype(float)
merged['pcw_published'] = pd.to_numeric(
    merged['pcw_published'], errors='coerce'
).astype(float)

# 2) Ensure anatomical_site is stringly‐typed
merged['anatomical_site'] = merged['anatomical_site'].astype(str)

# 3) Build mask for exactly pcw == 13.0
mask_pcw13 = merged['pcw'] == 13.0

# 4) Copy pcw → pcw_published for those rows
merged.loc[mask_pcw13, 'pcw_published'] = merged.loc[mask_pcw13, 'pcw']

# 5) Overwrite anatomical_site to "calvaria/skullbase" for the same rows
merged.loc[mask_pcw13, 'anatomical_site'] = 'calvaria/skullbase'
merged.loc[mask_pcw13, 'brc_code'] = 1111

# 6) Sanity‐check dtypes and a few rows
print("Dtypes:\n", merged[['pcw','pcw_published','anatomical_site']].dtypes)
print( merged.loc[mask_pcw13, ['Cell_ID','pcw','pcw_published','anatomical_site']].head() )


Dtypes:
 pcw                float64
pcw_published      float64
anatomical_site     object
dtype: object
                                  Cell_ID   pcw  pcw_published  \
141950  AAACAGCCAAATTCGT-HCA_BN_F13109714  13.0           13.0   
141951  AAACAGCCAAGTAAGC-HCA_BN_F13109714  13.0           13.0   
141955  AAACAGCCAGTTTGGC-HCA_BN_F13109714  13.0           13.0   
141956  AAACAGCCATAACGGG-HCA_BN_F13109714  13.0           13.0   
141960  AAACATGCAAATACCT-HCA_BN_F13109714  13.0           13.0   

           anatomical_site  
141950  calvaria/skullbase  
141951  calvaria/skullbase  
141955  calvaria/skullbase  
141956  calvaria/skullbase  
141960  calvaria/skullbase  


In [144]:
merged

,Cell_ID,barcode,status,assignment,Sample_ID,Donor,Sample_ID_pcw13,Donor_ID,pcw,Sample_ID_published,Celltype_fig1,pcw_published,brc_code,anatomical_site,Sample_ID_old_anno,Donor_old_anno,ANATOMICAL_SITE,PCW
0,AAACAGCCAACTGGGA-HCA_BN_F13177434,AAACAGCCAACTGGGA,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3
1,AAACAGCCAATAACCT-HCA_BN_F13177434,AAACAGCCAATAACCT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,DermFIB1,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2
2,AAACAGCCACAGGAAT-HCA_BN_F13177434,AAACAGCCACAGGAAT,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3
3,AAACAGCCAGTAAAGC-HCA_BN_F13177434,AAACAGCCAGTAAAGC,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN
4,AAACAGCCAGTTGCGT-HCA_BN_F13177434,AAACAGCCAGTTGCGT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,HCA_BN_F13177434,HHIP+PreOB,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191955,TTTGTGAAGGGTGGAT-HCA_BN_F13109719,TTTGTGAAGGGTGGAT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719,NaN,NaN,NaN,HCA_BN_F13109719,ChondroPro2,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2
191956,TTTGTGGCAGAAATTG-HCA_BN_F13109719,TTTGTGGCAGAAATTG,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719,NaN,NaN,NaN,HCA_BN_F13109719,SynFIB,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2
191957,TTTGTTGGTGATGGCT-HCA_BN_F13109719,TTTGTTGGTGATGGCT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719,NaN,NaN,NaN,HCA_BN_F13109719,ChondroPro1,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2
191958,TTTGTTGGTTCAAGCA-HCA_BN_F13109719,TTTGTTGGTTCAAGCA,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719,NaN,NaN,NaN,HCA_BN_F13109719,MaturingChon,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2


In [145]:
pd.set_option('display.max_rows', None)
dfs={}
for sample in merged['Sample_ID'].unique(): 
    dfs[sample]= merged[merged['Sample_ID']==sample].copy()

In [146]:
pd.crosstab(dfs['HCA_BN_F13109715']['assignment'],dfs['HCA_BN_F13109715']['brc_code'])

brc_code,1111.0,2372.0
assignment,,
0,5901,0
0/1,236,0
1,386,204
1/0,327,0


In [147]:
pd.crosstab(dfs['HCA_BN_F12627470']['assignment'],dfs['HCA_BN_F12627470']['pcw_published'])

pcw_published,5.8,7.6,8.3
assignment,,,
0,0,2870,1
0/1,0,2,189
0/2,20,74,0
1,0,0,1876
1/0,0,0,1
1/2,0,0,6
2,5212,0,0
2/0,159,1,0
2/1,1,0,208


In [148]:
pd.crosstab(dfs['HCA_BN_F12627470']['brc_code'],dfs['HCA_BN_F12627470']['pcw_published'])

pcw_published,5.8,7.6,8.3
brc_code,,,
2368.0,0,0,2281
2369.0,0,2947,0
2370.0,5392,0,0


In [149]:
pd.crosstab(dfs['HCA_BN_F13109715']['pcw_published'],dfs['HCA_BN_F13109715']['assignment'])

assignment,0,0/1,1,1/0
pcw_published,,,,
11.2,0,0,204,0
13.0,5901,236,386,327


In [151]:
merged['pcw'].value_counts()

pcw
13.0    19561
Name: count, dtype: int64

In [152]:
import pandas as pd
import numpy as np

# ─── 0) Setup ───────────────────────────────────────────────────────────────
# `merged` is  per‐cell DataFrame with at least:
#   ['Cell_ID','Sample_ID','assignment','brc_code','pcw_published',…]
merged['assignment'] = merged['assignment'].astype(str)

# ─── 1) Helper to pick the most common non‐null value ───────────────────────
def pick_mode(series):
    m = series.dropna().mode()
    return m.iloc[0] if len(m) else np.nan

# ─── 2) Build cluster‐level consensus of donor & age ─────────────────────────
cluster_meta = (
    merged
      .groupby(['Sample_ID','assignment'], as_index=False)
      .agg({
          'brc_code'      : pick_mode,
          'pcw_published' : pick_mode,
          'anatomical_site' : pick_mode
      })
      .rename(columns={
          'brc_code'      : 'Donor_final',
          'pcw_published' : 'Age_final'
      })
)

# ─── 3) Flag ambiguous samples: those where all clusters share the same pair ─
#  3a) Compute how many unique (Donor_final,Age_final) per Sample_ID
cluster_meta['pair'] = list(zip(cluster_meta.Donor_final, cluster_meta.Age_final))
n_unique_pairs = (
    cluster_meta
      .groupby('Sample_ID')['pair']
      .nunique()
      .reset_index(name='unique_pairs')
)

#  3b) Samples with only one unique pair are ambiguous
ambiguous = set(n_unique_pairs.loc[n_unique_pairs.unique_pairs==1, 'Sample_ID'])

#  3c) Annotate cluster_meta with a boolean flag
cluster_meta['ambiguous'] = cluster_meta['Sample_ID'].isin(ambiguous)

# ─── 4) Merge the cluster‐level metadata (and flag) back onto every cell ───
mapping = cluster_meta[['Sample_ID','assignment','Donor_final','Age_final','anatomical_site','ambiguous']]

In [153]:
mapping

,Sample_ID,assignment,Donor_final,Age_final,anatomical_site,ambiguous
0,HCA_BN_F12627470,0,2369.0,7.6,knee,False
1,HCA_BN_F12627470,0/1,2368.0,8.3,nan,False
2,HCA_BN_F12627470,0/2,2369.0,7.6,nan,False
3,HCA_BN_F12627470,1,2368.0,8.3,knee,False
4,HCA_BN_F12627470,1/0,2368.0,8.3,nan,False
5,HCA_BN_F12627470,1/2,2368.0,8.3,nan,False
6,HCA_BN_F12627470,2,2370.0,5.8,knee,False
7,HCA_BN_F12627470,2/0,2370.0,5.8,nan,False
8,HCA_BN_F12627470,2/1,2368.0,8.3,nan,False
9,HCA_BN_F12627471,0,2369.0,7.6,hip,False


In [154]:
# make sure assignment is comparable—if it's an int column, use 1; if string, use '1'
mapping['assignment'] = mapping['assignment'].astype(str)

# define your boolean mask
mask = (
    (mapping['Sample_ID'] == 'HCA_BN_F13109715') &
    (mapping['assignment'] == '1')
)

# assign both columns at once
mapping.loc[mask, ['Donor_final','Age_final']] = [2372.0, 11.2]

# verify
print(mapping[mask])


           Sample_ID assignment  Donor_final  Age_final anatomical_site  \
47  HCA_BN_F13109715          1       2372.0       11.2             nan   

    ambiguous  
47       True  


/tmp/ipykernel_3212985/1013632288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping['assignment'] = mapping['assignment'].astype(str)


In [155]:
mapping

,Sample_ID,assignment,Donor_final,Age_final,anatomical_site,ambiguous
0,HCA_BN_F12627470,0,2369.0,7.6,knee,False
1,HCA_BN_F12627470,0/1,2368.0,8.3,nan,False
2,HCA_BN_F12627470,0/2,2369.0,7.6,nan,False
3,HCA_BN_F12627470,1,2368.0,8.3,knee,False
4,HCA_BN_F12627470,1/0,2368.0,8.3,nan,False
5,HCA_BN_F12627470,1/2,2368.0,8.3,nan,False
6,HCA_BN_F12627470,2,2370.0,5.8,knee,False
7,HCA_BN_F12627470,2/0,2370.0,5.8,nan,False
8,HCA_BN_F12627470,2/1,2368.0,8.3,nan,False
9,HCA_BN_F12627471,0,2369.0,7.6,hip,False


In [157]:
# Make sure both keys are the same dtype
merged['assignment'] = merged['assignment'].astype(str)
mapping['assignment'] = mapping['assignment'].astype(str)

# Now merge Donor_final and Age_final into merged
merged = merged.merge(
    mapping[['Sample_ID','assignment','Donor_final','Age_final']],
    on=['Sample_ID','assignment'],
    how='left'
)


/tmp/ipykernel_3212985/1274010713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping['assignment'] = mapping['assignment'].astype(str)


In [160]:
merged.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_barcodes.csv')